### 1. 대화내용 불러오기

In [1]:
import pandas as pd
import numpy as np
import pickle
from pprint import pprint
import re

In [2]:
with open("data/cleaned_data.pk", "rb") as f:
    data = pickle.load(f)

data.reset_index(drop=True, inplace=True)
print(data.head())
print(data.info())

             Date User                Message
0  2018. 5. 3. 00  주현                엥 결국 안먹음
1  2018. 5. 3. 00  주현    너 그 야구부 엠티는 가는거야 그래서
2  2018. 5. 3. 00  현영                      안가
3  2018. 5. 3. 00  현영        근엄마한테갈거라고말해놓긴햇는디
4  2018. 5. 3. 00  현영                  보내준다허면
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 22914 entries, 0 to 22913
Data columns (total 3 columns):
Date       22914 non-null object
User       22912 non-null object
Message    22914 non-null object
dtypes: object(3)
memory usage: 537.2+ KB
None


### 2. 초록 없는 데이터 제거  및 분석시기 설정하기

In [3]:
# 시간정보 열 datetime 정보로 변환
data['Date'] = pd.to_datetime(data['Date'])
# 인덱스 넣기
data = data.set_index('Date')
data.head()

,User,Message
Date,,
2018-05-03,주현,엥 결국 안먹음
2018-05-03,주현,너 그 야구부 엠티는 가는거야 그래서
2018-05-03,현영,안가
2018-05-03,현영,근엄마한테갈거라고말해놓긴햇는디
2018-05-03,현영,보내준다허면


#### 연도별 대화내용 분리 (DTM 분석을 위해)

In [7]:
year2018 = data['2018-01-01' : '2018-12-31']
year2019 = data['2019-01-01' : '2019-12-31']
year2020 = data['2020-01-01' : '2020-12-31']

In [8]:
# 제대로 분리되었나 확인
print(year2018.head())
print(year2019.head())
print(year2020.head())

           User                Message
Date                                  
2018-05-03  주현                엥 결국 안먹음
2018-05-03  주현    너 그 야구부 엠티는 가는거야 그래서
2018-05-03  현영                      안가
2018-05-03  현영        근엄마한테갈거라고말해놓긴햇는디
2018-05-03  현영                  보내준다허면
           User                Message
Date                                  
2019-01-01  지이    얘두라 따랑해 새해복 많이받아❤️☺️
2019-01-01  현영             사랑해 민나상 모두들
2019-01-01  현영                    오메데또
2019-01-01  현영               앗 언니자화상이다
2019-01-01  지이                     졸귀탱
                    User        Message
Date                                   
2020-01-01 00:00:00  현영            고구마 
2020-01-15 18:00:00  영현             늼들아
2020-01-15 18:00:00  영현    우리 설끝나고 모이자 
2020-01-15 18:00:00  지의             그러자
2020-01-15 18:00:00  지의              제발


In [9]:
print(year2018.info())
print(year2019.info())
print(year2020.info())

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 5991 entries, 2018-05-03 00:00:00 to 2018-12-31 23:00:00
Data columns (total 2 columns):
User       5991 non-null object
Message    5991 non-null object
dtypes: object(2)
memory usage: 140.4+ KB
None
<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 13858 entries, 2019-01-01 00:00:00 to 2019-12-31 23:00:00
Data columns (total 2 columns):
User       13856 non-null object
Message    13858 non-null object
dtypes: object(2)
memory usage: 324.8+ KB
None
<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 3065 entries, 2020-01-01 00:00:00 to 2020-06-17 17:00:00
Data columns (total 2 columns):
User       3065 non-null object
Message    3065 non-null object
dtypes: object(2)
memory usage: 71.8+ KB
None


##### 시기별로 나눠서 데이터 저장하기

In [10]:
# time_slice별로 데이터 갯수를 저장할 리스트 생성
time_slice = []


slice0 = list(year2018['Message'])
slice1 = list(year2019['Message'])
slice2 = list(year2020['Message'])

# LDA 분석할 때와 비슷하게 모든 분석 대상 텍스트를 담는 리스트 생성
tokenized_data = [msg.split() for msg in (slice0 + slice1 + slice2)]

# 각 slice에 들어 있는 갯수를 원소로 갖는 리스트 생성
time_slice.append(len(slice0))
time_slice.append(len(slice1))
time_slice.append(len(slice2))

In [11]:
print(len(tokenized_data))
print(time_slice)
print(slice0[:5])
print(slice1[:5])
print(slice2[:5])
pprint(tokenized_data[:5])

22914
[5991, 13858, 3065]
[' 엥 결국 안먹음', ' 너 그 야구부 엠티는 가는거야 그래서', ' 안가', ' 근엄마한테갈거라고말해놓긴햇는디', ' 보내준다허면']
[' 얘두라 따랑해 새해복 많이받아❤️☺️', ' 사랑해 민나상 모두들', ' 오메데또', ' 앗 언니자화상이다', ' 졸귀탱']
[' 고구마 ', ' 늼들아', ' 우리 설끝나고 모이자 ', ' 그러자', ' 제발']
[['엥', '결국', '안먹음'],
 ['너', '그', '야구부', '엠티는', '가는거야', '그래서'],
 ['안가'],
 ['근엄마한테갈거라고말해놓긴햇는디'],
 ['보내준다허면']]


### 3. Dynamic Topic Model 돌리기

In [12]:
from gensim.models import ldaseqmodel
from gensim.models import CoherenceModel
from gensim.corpora import Dictionary, bleicorpus
from gensim.matutils import hellinger
from gensim import corpora
from tqdm import tqdm_notebook
from time import time

import os

##### dictionary와 doc2bow 만들기

In [13]:
# Create Dictionary
if not os.path.exists('kakao(DTM)_dict'):
    dictionary = corpora.Dictionary(tokenized_data)
    #dictionary.filter_extremes(no_below=5, no_above=500)  # 이 줄의 코드는 n회 미만 또는 n회 이상을 지울 때 사용
    dictionary.save('kakao(DTM)_dict')
    print(dictionary)
else:
    dictionary = Dictionary.load('kakao(DTM)_dict')

# Term Document Frequency (convert tokenized documents into a Document-Term Matrix)    
if not os.path.exists('kakao(DTM)_corpus'):
    corpus = [dictionary.doc2bow(doc) for doc in tokenized_data]
    corpora.BleiCorpus.serialize('kakao(DTM)_corpus', corpus)
else:
    corpus = bleicorpus.BleiCorpus('kakao(DTM)_corpus')

In [14]:
# dictionary함수로 만든 사전에 있는 단어들 모두 볼 수 있는 기능
print(dictionary)
for idx in dictionary:
    print(dictionary[idx])

Dictionary(24575 unique tokens: ['결국', '안먹음', '엥', '가는거야', '그']...)
결국
안먹음
엥
가는거야
그
그래서
너
야구부
엠티는
안가
근엄마한테갈거라고말해놓긴햇는디
보내준다허면
개이득
파리타임
마커롱
먹고싶따
마카롱임
바보야
야웃김
고딩때보내줫는데
내
내친구가
리얼
얼굴이
프라이팬
아개웃곀
그린줄
동그라미임
컴퍼스로
뭐지
반응은
고딩때야
개똑같아
지금이랑
귀엽지
얘그라
이거어때
똑같다니
야
귀여오
시부랭
이주희
작가임
에바참치쌈싸서
저얼굴이면
지금도
한입에털어넣야됨
털어넣으셈
근데
똑같음
참치쌈싸서
털어놓어야함
똑같다고
레알
개충격
시방
컴퍼스
잠깐만
레알똑같넼
시부랰
진현영대잔치
아닠
다른점
떡지고
안떡진고
앞머리가
개좋앜
앜
미틴
존나
행복하다
깜찍
왴
개욱겨
아근데
널
좋아해버렷
똑같이
생겼는데
똑같으면
쌈싸목는다
에바
쓸거얌
으로
저거
진짜
레알똑같을줄은
아니
자가
겟아웃오브히얼
이거
잘못하더거
단톡에
데사
보낼뻔
보내지
아
죽을뻔
죽일뻔
보내면
진현영
자퇴할까봐
휴학신청서
끄적
야주희
몰겠눙데
물어봐도되나
이런거
걍
물오봄
사귀제
정우철이랑
헐
눈치로
다아네
뭐야
나만눈치가
준내없엇군
언제부터야
준내
2주됫데
왜비밀로함
나도며칠전에앎
몰라이제
거의
공개임
애들다알어
나만몰랐쓤
나도몰랏음
야넌근데
17에도
몇명은
안댕
안다
난
말안해줙음
이걸
평생
이제물어본다고
썸인줄
젤늦게암
그리고
대답
내가할랬는데
니가말해
왜
언니그때
나
이언니
주현
화욜날
쉬는시간에
씨
웅웅
편의점에서
고백언헌더고
정우철한데
냐가
지랄할때
씨쉬는시간을쉬싸는시간으로봄
다웃었잖아
옆에
맞아
그때
말안해주고
배창현이
그썰들었다
지의야
너무허다
눈짓으로
개웃기네
보는맛에
조용히
하라고
함
비밀로
사겼지
개사악해
미친
소리나
다웃는거
그때는
몰랐는데
긱사와소
생각하니까
뭔가
이상하길랴
웃곀
정우철카톡상메보니까
되있길래
생일축하해
사귀는가
이랬씀
그것도
그럴수잇지
보면서
좋아하니까
이럼
드라마썼구나
헌자
스뱔
1도없어
진짜눈치
도토리묵임
현영이

원주민
흑인
생김
한대치고싶게
누군지
몰라도
정우철
마즘
아니샹칼
존똑띸
니들
창토라
http
같이사실분
사싀른
페북대란템이거든
111
진행중이거든
밀착력과
커버력
동시에갖췄거든
좋음
닥터자르트
라인계속탈래
방귀
쳇
프사잘나왔다
오늘그소리만몇번짼지
머쓱
나눠써
리필이랑
본품이랑
주는데
리필가질게
헤헷딴데끼워씀되징
야야
성적나옴
봐봐
특히
개빨리나와
없었나봐
채점할게
강의평가
되는거아냐
하면
88
왜안나와
개좆가틈
의견등록했는데
안되지
다들나옴
의견등록하구
밑에
주관식도
해야돼
주관식이
객관식임
주관식말구
나왴
동거동락해야지
성적메이트
알려줘
못봐
알바끝나서
지슴
세종
학사정보
들어가야해
아머야
뭐나왔써
개행족해
디
목표고
씨가
예상이었는데
키키
비나옴
제로
비쁠이라도
소박한행복
애들다
행복해
비제로
개만족
비플이라고
씨플
진현영이랑
씨플나와소
재수강아니라고
좋아했는데
비나왔네
씨플이어이뎌
어뎌
개궁금해
물리학점
믿는다
날실망시키지말기를
너만큼은
잘봤거드
뭔데빨리
현기증남
집와서
곧확인할게
b이요
지렸다
에이스
역싀
중간고사
0점
개웃갸
아나
가능해
몇점인데
0점이
아니라고
5점
50점
49점인데
종강
1점만
더맞으면
될수있었던건깡
쁠
과제나
실험에서
영현
깎인거아님
깎였나보다
못본
오지게
퀴즈
비쁠메친
개부럽당
똑같앸
언니나랑
물리메이트하쟈
개높아
퀴즈점수가
0
좋아좋아
높은데
퀴즈는
c떠라제발
미적나옴
안나옴
물리재수강아니다
살았다
못나올삘
물리는
오히려
재수강하면
개어려웠어가지고
불안
이번에
개망함
내학점
미적근데
다어렵대
F각
미적도
F각이야
미적은
가럼
영화인보고
마셔
술만
안볼것같앵
옹키올티
실화야
학정
밥먹으러
정영현이랑
학교왔는데
김민준한테말해
개뚜드려맞았잖아
김민준이
말했어
앝무한루프오졌네
갈사람
건대쪽에
쇼핑
이쑴
기말점수나옴
님덜
뒷북인가
나온댕
월욜에
학점은
아근데왠지보기싫다
으앜
지냄
끄앙니뜰
축구봄
보구있었으
한골먹힌거
축구재미없더
규냥
자려궁
대한민국
대한미국
짝짝짜까짞짞
짞짞짜짜까까
술마셨어
티비에서
소리들려서
열기에
취했어
다보나봐

뼈
수준이야
동참하실분은
일어나주세요
청자켓까지
매너봐
벗어주는
지꺼야
다님
박도혀오빸
청자켓입고
벗어줌
이요르
벗어준거라고
미이친
미친미친미친
없슴니다
와우
친구사인데
거울셀카까지게다가
그것을
벗어주고
청자켓
프사로
남녀칠세부동석의
원칙에따라서
이것은
아니하다
무기징역일세
생각으로
이상한거야
남자사람친구가
옷벗어줄수있씀
추우면
그랬지
뭐라
올리는사람
승헌쓰
유투브에
폴킴폴킴
오키염
정주행하시오
모하슈
밥하고이썽
나갈준비쓰
어디가
가쟹
코다차야
조지고올게
치즈야
김볶으로
유명하길래
먹구있어
빈곤하게
이런거하자나
홍보알바
건수
잘땡기는중
거기
관둠
얘기해줄겡
아낰오키
난빙수먹으러갈건데
서울빙수추천좀
설빙
빙수먹고싶다
개맛잇어보임
김볶
쉬부레방구
어때
후기좀
개존맛탱
같이가자
야한거야
개야해
각인디
야동이라매
긱사안드감
집감
놔두고
멀쩡한
어디서잠
노노
밤샐라고
주희어대
어대
탐탐
보리잉줄
귀욥다
짱
왜저길올라갓냨
햄스터가
그러고보니
교수님
수염짜름
들었나
말하는거
미스터선샤인
종방
꼽준
닮지않음
꼽줬어
신발
우산이랑
챙기기
공지
톡게시판
메모장
똑같다
사는거
모햐
보구옴
쇼미더머니
갓겜
쇼미달려야지
오늘밤
퇴근
뺏기는중
필기
베끼는중
누가뺏엌
매일밤
안부물어보넼
윤지의는
물어보고는
아무대답없음
내말잌
놀러
델꼬가
둥둥둥이
둥둥둥두루둥둥이
배경이랑
프로필뮤직
공차사이즈업해서
지하철역에서
마감할인하길래
빵
나도모르게
이끌려서
빵먹고
미친년
이제는
이별을
선언한다
놨는데
헬스장가보니1키로쪄잇엇음
쉬부레
이틀만에
메로나도
쓴고임
내자신에게
약속겸
퀴즈결과나옴
건대에
개콩
떡볶이
콜
안자구
머거봐써
맛있음
👍
엽떡보다더
안좋아해서
엽떡별로
그런느낌
신전파
엽떡보다
매움
엄청난
정돈아니고
맴단계
안맵고
있을걸
정할수
떡볶이야
맛있는떡볶이
느낌이면
늘러붙은
엽떡이
퍼지고
생생한
쟨
표현죽이네
와윤지의
입맛
존떡이냨
지각임
존에
쫌
긱서임
김
나아직
지각
통계학개론
도착안함
이제출밡
같음
사투리
북한말같음
티안남
사투리라기보다
어눌
그런건가
참는다고
과
보내줬는데
애가
애하는

로션발랏는데
세수하고
개따가워
미침
어쩌다
저래됨
면도하다가
울집오니까
해달라하지
면도기로
에
그건아니구염
말하라햇지
윤지의한테
해달라고
눈썹칼삼
다이소엘서
흙흥
뽑지
눈썹미는데
눈썹칼로
핀셋으루
피가나
뽑는건
조또무서워
월욜날
시험치는거
맞나봄
시발
시험이요
sw
뭐시기
뭔
시험
못봐도
상관없잖아
미쳤나
헤이즈오는대
그게문제가아니고
헤이즈요
보고싶어
헤이즈
몊시야
10분까지
6시
입실하라던데
31일이라더니
왜죠
필수야뭐야
신청했으면
시험쳐야됨
왓을걸
접수됏다고
언니햇너
신청하라고
당부하던데
나신청안햇는데
신청하라그래서
했움
나왜안알려줫어
얘기했을걸
한것같은데
바보
두시에
늦지말고
및친
나빼고
다신청햇음
없나
학번만
안해두되는거임
결석처리되는걸걸
장학금준다고그랬음
못받겠지만
개꿀이러구
개꿀이야
꿀이지
받을수
있어야
슬퍼
개꿀이잖아
장학금
깔아주는
사람들이야
결석처리이런
씨방
무슨요일
결석처리
교수님한테
말씀드리면
될거야
현영쓰
욕을
먹지않을끼
안보면안돼
안보고
블랙보드에
빨간색으로
생리공결
무조건하라고
내면안됨
친구학교놀러갈건데
괜찮은방법인데
으음
시발시발
5분만에
나오면안됨
잘볼샌각없음
인증시험때처럼
시밬
1시간은
쳐야하는가아니냐
암맬도
없는디
없어졌나봄
2차인증시험날짜가
갑분싸
오져버림
떳음
개그지같다
1차인증
2차인증
바로다음날
예고하는
문정씨
같아
살인예고
욕쳐먹을지
상상도안간다
꼴등
꼴등이야
그거세지
빨리준비하라
살인예곸
첫날
운갈래
아음성지원
개싫엉
눈썹피보여주면서
슈불
잔소리하면
피보고싶지않으시면
그만하세요
한마디할거임
무섭넹
윤지의가
생각낰
개웃겼어
따라했던거
밤새떠0
왜새
샐
생각을
이틀연속으루
갈수있겠엉
쿠쿠쿠쿠큐큐큐큐쿠쿠쿠큨
쿠쿠쿠쿠큐큐큐큐큐큐큐큐큐큐큐큐큨
만날때까지
생각즁이야
안잘까
만나고
어떡할라구구
개고민됨케
밤새
피씨방있었어
밤안샜는데도
다섯시에나왔는데
한시간만
나왔으면
다섯시에
자지
조금이라도
일어날수있을까
생각하지말구
일단은
깨워줄게
한시간뒤에
미치겠
와너네
일찍일어난다근데
일어나
깜짝놀럈어
깬건가
오늘만
1시로
미뤗어
자러갑니다

하자하자
메세지
보내떵
말해쥬
할사람
4월
오전이면
엔터식스갔넹
하냥대
갔는데
사려고
피규어
개비싸서
구경만
수업이떠
오전에
강주현바쁘게
다니는거봐
지땨
글쿠만
땜에
못찍겠다😭😭
널구있어
띵가띵가
저게뭐유
페북태그했자너
페북알람이안왘
보린데
얘도
보리랑
완전다른데
더예뻐
쟤가
더귀여워
우리보리가
너네오늘도
쥰내배고파
두식이랑
안날뛰누
꼬리털
고것만
유튜버접수함
교복입음
100만
유투버임
질올
질올이뭐냐고
놀리겠지
아싸네
개아싸
샘나누
유투버
오른팔걸께
정현이가
찍었다에
최소
댕강
말해봐
사실이야
정확한듯
지룠다
깜짝놀랬네
무슨얘기결론
1명
대답하는사람
물은사람0명
쉬밬
아니대답좀해달라
이거를
늘려서
말하네
몇배로
결론은
자기도
자신감을
키웠다
결론
최종
can
do
it
you
I
because
학교탈출
개아깝다고
생공쓴거
붕어빵
자색고구마
지금의
행복
먹고오세유
아침을
젭알
졸려
개잔와소
안씻고
출튀할까
잠깨
카톡보내다
출튀한다
출튀할바에
메일보내삼
얘드라수
가는게나음
안오고있잖아
들어가기좀그런데
출첵함
밖임
딱히안와도
미친근데
낮어서들어가기
늦게와
맨뒤에앉음
강의노트보면
이해가능
들어가기
뚫고
미앙
민망쓰
진혀녕빼고
맨뒤
앉아버리기
만나요
무슨날이가
늦었노
8시에일어나서
화장다함
1교시라서
개쩌네염
수업안듣고
화장함
웹프
메일보낼까
어떠케생각하니
개피곤
아사람
언젠지
우리무슨
있었지않냐
참여하고
휴강하는거
뭐했냐
웹프시간에
내일이라했잖아
웹프교수님이
휴강임
그렇다더라
참여해야지
휴강인거야
강의들어야됑
그대신
참여안하면
강의가
1도몰르는데
긍까
내일임
자료구조안가
12
오라카려나
시릇
휴강한데
개행복이데
빠지면
자료구조
실습안해도됨
개꿀딱지
들어야함
들으러
신청만할거임
마쳐
12시
이고
고를꼐
뭐냐
고르세여
사무부
입금해
송준규씨
님들아
과제에서
이부분
내부
참조라는말이
저건또
눌럿을때
어디다른페이지로이동하는게아니라
같은페이지않에서
또있씀
웹프과제
이동하도록
하는거냐
이번에한거
저거뭐엿지
눌렀을
뜨게하는건가
페이지
internal
reference면
다른페이

...

현웃터졌다
눈치챘겠지
카톡하는애
첨봄
깔아
보림
직접깐거임
심심하니까
윤지의자리네
저기이제
센에
깜
몇개
좌석에
깔릴
예정
다운되더라
1개월만
기다려보렴
다음사람
네일까쥐
영현이원피스에
호피아우터에
뒤집어지네
키보드소리
세상개큼
걸어다니는
내손톱길이
짐승이라고
다시해야됨
성격이
여자라서
잘마즘
빨간부분이
오빠얼굴이고
너얼굴인줄
노란부분이
다른거네
전혀
사람도아니네
그니가
의자냐
사람도아님
나이거자꾸
flex
로
읽어짐
사람얼굴인줄
표현과
플렉스
후진오차
오차
후진
단톡이
빨라짐
어려워
질수록
후져버리기
이공식
중요해
반비례관계
간지도
거까지
지금알았네
진도
실제상황
길가다
버정에서
주암이
다르다아님
앞문후문
후문이여
주암이아닌뎅은뭐냐
갓대
실종되서
왔음여
우리는아까너가실종됫다고
수군수군
이수군
댓음
친구추천알람떠서
봤는데
도랐냐고
말잇못
저티
맞춤이가
좆되네
영덕서의
거꾸로
흐른다
넣어놨는데
한줄
낼학교에
워크맨
개웃김진짜
이데
머리말료
배고픈사람
고프진
않당
갑자기고파
모의토익
신청하세융
줏음
7이야
8이야
사랑하고
칼답
식사중
다먹고
봐줄수있낭
민준이오빠랑
열띤토론중이라
딴애들
봐주겠지
영현이나
딴애들봐주겠지
주희나
즂같다
27500원
6875원
맛잇겟따
가발
지의의
포니테일
보야줘야할지모르겠어서
찍어봤써유
눈너무
무서운데
하고오나
일찍일어나면
아침부터
옶어서
언니같다
힘들거같은데
똥손은
절대못한다
저얼굴이라니
화장안지윤거야
화면봐야했냐고
줜나이쁨
별걸다사네
가슴노래
다보내주는중
난에거
이거보내면되냐
이제는꽉찬가슴
개많은뎊
풀버전
노래들이
띵곡임
꽃보다남자
존나띵곡인데
띵곡의
참뜻
알겠니
드실
나심심해서
이러고있었는데
분유사주실분
분유퍼먹어줄래
언제다씀
와저거
만항다
많았다
아재머임
넥스트라면맛집
넥스트
존나맛잇음
알놈알
넥스트가
피시이름
지겨워
없늬
마스카라있는사람없지
놋북있지
마스카라는
마스카라
안해본지
오래되
존내
아직한다고수업
알람이랑
다섯통해줬는데
전화
공결쓴다
12통했네
갈껩
공결쓰고
마춰서
쉅마칭때
언제드갔는데
45분에
끝납니다
저희
감사합니다

몰리
33일껄
어디서봐써
33이
맞다면
B110
두드림에
화면끔
누구확안래줘
742도
확인
영현잌
센B201
dh
와맋
같은곳이다
역쉬
광이네
아레알
528개
광
싫
헬
미리가있자
습팔
에휴
드가자마자
나가야함
퇴실시간
일곱시부터지않나
그랬던거같은데
알바사장님한데
늦는다구
너네사장님
말하렴
보내줄듯
그거때매
뺐윰
수욜로
점수나옴
줬대따
좆도네
아존나보기싫어
없었쑴
sk
Fdlsep
F라구
너무실망하지
않으시길
말만을
믿겠어
나온다는거지
cook
90A
g
볼까말까
개고민중
블랙보드지
uis
있는점수는
엣
아나는
점수다르긴해
낮긴해
블랙보드가
더낮고
높게나오는데
블랙보드는
출석점수더해진건가
uis가
학사정보시스템
신입생이어서
쥐긴넹
몰랐돈거
15분까진데
열시
애이득
하하하하
티내지마
헷갈리셨0
닥쳐
쌰랍
나가면
한명이라도
재개
빨리나가
누구라도
반까진데요
저희수업
죽는다
죽여버릴다
했지
다시잠
안오니
안온데
하기시러
아들얘기
안궁금해요
이런거쓰면
신상털림
아이패드
자랑그만해주세요
누가쓴지알수있나
집요하게
털라면
털수도
마치구
과방에서
닭도리탕이랑
먹구가
닭도리탕
4인분
시킴
최고다
잘먹겠숩니다
올수
있지
아쉽•••
4명인줄알고
기회를
놓치다니
오늘같은
4인분시킴
난개꿀
와오빠
대박이네
맛있는거먹는중
오메
뭔디요
부럽
뭐먹는데1
bab
jib
what
닭도리탕까진아님
먹방쫌
찍어조
고추장돼지불고기
대리만족
닭도리탕아님
찜닭임
shit
개꿀맛이겄네
자체휴강이
승자다
최애메뉸데
모의토익땜에
학교감
닭도리탕아니고
개최
대회
오늘교수님
제목짓기
패션
아이디어
내주세요
야나도
똑같은생각
햇는뎈
고추장
불고기인가
지퍼가
고장불고기
도발적임
아주그냥
옆에도
여는순간
지퍼를
열면
촤악
속옷인가
아개궁금해
도발적이다
On2일듯
최악은
헤헷찡
닭도리탕맞음
찜닭양
먹는다매
찜닭아니냐
빨간거
찜달깅라길래
매콤한게
빨간거라며
둘다좋음
맛잇겟당
끝났네
제목짓기대회
이거찾아왔는데
열정
아홉시부터
생각하고있었다곸
찾은거냐고
가운데
지퍼까지
세심하다
미기
시타
히다리
마에
우시로
이시국에
매국

쳐볼라고
미친뇬이
말이안나오는
철면피
맞짱뜨고싶다
관련도
배우는거랑
없는걸
넣냐고
점수에
개썅뇬
계약하고
싶은사람만
하면되지
시켜
지아들
뭔죄
아들
안듣고
우리얘기
있는줄알았는데
있었음
유주연한테
조언받으니까
기분나쁨
내얘기도
우리얘기하는데
조언도
끼어들어서
딸도없으면서
해준다는말이
그래요
어쩔수없대
엄마입장에선
공감능력
상실했누
맘카페나가라해
받아주기
힌들었다
표정관리
개썩었었음
나자꾸
너봤잖아
끝내
않던
올라가지
입꼬리
올라감
한쪽만
발표저거말한사람
더말해줘
사이다다
시원
편안
킫밀리온댜
나무조건갈래
가야돼
수업빼고
양띵도
오넼
똥고집
zzzzzzzzz
좆같
까만하트
써놨는데
인스타이름
저거없었는데
좆같은데
따라한거
그런데
내생각에는
차단박을까
저거임
JUHEE
🇲🇽
🖤
이거였음
겹치기싫어서
바꿨어
교수님이야
새로
오신
웅노답교수
always
수고
좆됬당
지금이라도
약임
전남친
차단이
퇴치에서
꿀팁감사염
백빵따라햇네
검정하트를
쓴단말임
어느
말듣거
슨배님
오는길
차단박고
검정하틐
난봣다
왜지웤
말넘심인가
개별루다
낰
알았냐
얘긴줄
전남친이
집착하는거
개불쌍하노
검정한번썻다고
민석이
또쓰면
민x이처럼
차단당함
민x이
나도사야지
에어팟사러간당
규어팟은
규보에게로
새거사
kt샵에서
결제하고
말해서
중고나라
함만
미개봉으로
중고나라에서
중고나라에서삼
검찰공무원가족입니다
직거래했데서
직거래가는즁
싸지두
않네
몇마넌
안나네
차이
옹근데
최저가일걸
2세대
중에선
아니라서
중고도
실리콘케이스
철가루스티커도줌
카라비너
카라비너는
고리같은거
15마넌이먼
앵간한가
2세대니깐
앵간한거아닌가
1세대랑
별차이없지안나
16만원쓰
사면
케이티샙에서
사꾸야
만원이라하면
십사만원
에어파
증정품따로
합리화해보리기
드디어산다
귀찮게
덜한다
드디어드디어
필요없지
넵
일련번호확인할게여
위에분
개걸렸쥬찔려서
나오는것
동문서답하쥬
속마음
채점절대
개이득이야
돼서
안찍어도
김에
만땅해서
비빔국수
셀프리필해서
웹툰봄
지중해4팀옴
출근한
폰충전만땅
일하는거보다
더힘들듯
왔다갔다가
4시간반동안서서
받아왔는

하비않나
엽주고뭐고
염증난듯
쉬발
죤나배아파몇일동안
코로노아니제
감기증상••
엽주는근데
요번주의
사투리냐
아프겠다
개아픔
똥은안나와
그놈의폰부터
수리햌
살빠지겠노
괜찮아진거같으면
맛있는거쳐먹고
몇일동안
아팠다가
어랏괜찮나
어제도오빠랑애슐리먹고오늘도방이샵샤브쳐먹
또아파
이런썅
아프다돈데
그만먹고
약을먹어
그만먹곸
더먹자
약을
어랏괜찮나할때
아프겟다
잘잠
잠은
고통이
그나마잊어짐
잠자면
잠깨면
죤나또아픔
너네근데
휴학신청했니
2월7일까지돈데
깜빡해서
학교다니게되는
왠열
기대했는뎅
어캄을
꿀맛
찐
몬데
카라멜팝콘
영화보고
클로젯봄
네일아트
또하셨네요
흰죽••
노잼
클로젯
멋잇러보임
저것마저
판타지
뛰쳐나갈뻔
요즘들어
최고노잼은
미스터주임
개쓰레기영화
그게머임
몽미
다봐서
저거까지봤더니
잠만자다나옴
클로젝
잼없슴
클러젯
기대중인데
볼만혀
갑툭튀부분에서
다른사람들은
조용한데
정원이혼자서
재밌던데
으아아아아아악
오빠때매더놀램
어떤사람이
영상찍어서
유튜브올렸는데
썸네일부터
개좆가탘
신고때릴거야
개어이없어
미친메친
그와중이뿌다
신고해줄까
뇨뇨
유튜브
개깐깐해서
신고
본인만
넘고맙
마음은
찾은거도
알려쥼
일하는
도라인가
개이뿐데
눈대박많이와
누니와
개추워
탈색에바
샛노랑
은데요
이쁠거가
답정너여써
눈조와😍
같운뎅
괜찮을
아이거진짜
봤다가
시뱅
호기심에
후회중
이와중에
누텔라아깝자
똥몰카
토스트로
에바짘
마요네즈를
올리는건
개꿀조합임
잘먹을거가틈
나야
유튭
언니야
후련
님들님들
색깔
스페이스그레이
실버
은색과
회색•¥
무슨차이지
보여죠
고민된다
할거
케이스라
블랙계열도
실번데
원랜
이쁘던데
넣는
사긴살언데
아이패드랑
프로
상관없겠다
파우치면
얻음
그레이
까리해서
스페이스
하고싳음
매년
주는거잇는데
판다길애
나주라그랫음
괜찮은듯
고민끄
간지느는데
그레이가
안질리기엔
깐지나
스그가
스그사
임마
답정너엿음
스그
아니가
니가게에
있닝
두래
살려눈데
울집앞에
핑크랑
화이트
담아먹을라구
울오빠랑
생겼누
카톡안읽냐
죽을래
카톡했닝
운동가방
이고임
운동하니
개웃교
민수는
살언제빼냐
배꼽이
운

탐았다
오해풀고싶은데
말하긴
잘못없눈뎅
뭔느낌인지
오해를
파리지앵들의
풀고자
15명이나있는데
수치플
실수
아니에요
열허분
하고싶네
쪽이냐고
현영의
잘했더
쩔지
헤헷뚝섬사람
줫나줜나
날씨존나좋자나
나도놀러가는즁
멋있겠다
와개쩔어
어디가셈
서촌
멀리
뭐로만든거
입은
흰색
취주
치즈
앜마늘인줄
가고싶었는디
매늘은좀
우리한테도
해줘
어이쿠저런두번은모태
우리도시락먹고배1도안차서
황금올리브반반시킴
아니이거
뚜레쥬르
배달안되냐
빠바랑
쿠키는
받아봤는데
푸는
노가다로
머리로
풀거나
풀수
있더라고
좆됬음
중딩은
푸는방법이
그떄그때
다까먹음
공부해가야됨
선생님다신안함
짲응나서
후회된다
개쩌는게
우도에서
지갑잃어버렸는데
서귀포파출수에서
받기로함
노트북을
두고왔었다네
들고간
공항갔는데
찜질방에서
정진수
재택근무함
생각하면
진수탈출이
꿈만같다
다시금
행복함을
어케살았냨
살랑해요
후족발야시장
야요즘
잘되지않냐
따뜻해서
식당
잘된다는디
공항가는길에
공항가소
난리도
발견됬데
신분증없어서
아니었음
없어진고
우도
해수욕장에서
2팀
어이쿠
그돈
알바비로
주셨겠네
사장넴
안타까운
삼겹살먹을깨
스낵랩
쟁반짜장
스낵랩은뭐냐
디저트로
말한거지
엽떡먹음
목음
에엣
누구긴
비어서
오빠집에서시킴
배참
스낵랩3개
간고야
부타
삽겹살
한쵸
한표
짜장
노맛이라매
유튜버들도
애매하다함
오매개쉬
오아아아아아악
비계
잘못걸림
따흑
얼려놨더거든
한번먹을양으로
주방장같은데
제육도만듬
제대로네
강쥐같지않아
강쥔줄
아들아
팔도비빔면은
라면이야
딱말해
라면이지
비빔면이지
개빡치네
답은
있었던거야
정해져
내기해쏘
비빔면
계란후라이랑
맛있씀
식혜좋아햐는사람
식혜빡
이임
빠순이임
식혜한병주까
만드는거임
식혜맛임
대구갔다와서
줄께
가져갈래
달래도
개꿀맛임
달래
만들어서
비벼먹으면
양념장
콩나물밥에
넣어먹고
된장에
뿌리는
20대
캔거다
달래캠
쑥이랑
아앗
식혜
캔거냐곸
달래양념장비빔밥
지의집가서
양념장도
다가져가
밥빼고
안만들었냠
양염장
달래양념장을
왜만들어
안만듦
지의다시읽어보니
야영현집들이한지
이주지남
안만들었는데
양념장아직
뭘달

In [15]:
# 사람이 이해할 수 있는 형태로 코퍼스 사전 재구성 해보기 (term-frequency)
[[(dictionary[id], freq) for id, freq in cp] for cp in corpus[:]]

[[('결국', 1.0), ('안먹음', 1.0), ('엥', 1.0)],
 [('가는거야', 1.0),
  ('그', 1.0),
  ('그래서', 1.0),
  ('너', 1.0),
  ('야구부', 1.0),
  ('엠티는', 1.0)],
 [('안가', 1.0)],
 [('근엄마한테갈거라고말해놓긴햇는디', 1.0)],
 [('보내준다허면', 1.0)],
 [('개이득', 1.0)],
 [('파리타임', 1.0)],
 [('마커롱', 1.0), ('먹고싶따', 1.0)],
 [('마카롱임', 1.0)],
 [('바보야', 1.0)],
 [('야웃김', 1.0)],
 [('고딩때보내줫는데', 1.0), ('내', 1.0), ('내친구가', 1.0)],
 [('리얼', 1.0)],
 [('얼굴이', 1.0)],
 [('프라이팬', 1.0)],
 [('아개웃곀', 1.0)],
 [('그린줄', 1.0), ('동그라미임', 1.0), ('컴퍼스로', 1.0)],
 [('그', 1.0), ('뭐지', 1.0), ('반응은', 1.0)],
 [('고딩때야', 1.0)],
 [('개똑같아', 1.0), ('지금이랑', 1.0)],
 [('귀엽지', 1.0), ('얘그라', 1.0), ('이거어때', 1.0)],
 [('똑같다니', 1.0), ('야', 1.0)],
 [('귀여오', 1.0)],
 [('시부랭', 1.0)],
 [('이주희', 1.0), ('작가임', 1.0)],
 [('에바참치쌈싸서', 1.0), ('저얼굴이면', 1.0), ('지금도', 1.0), ('한입에털어넣야됨', 1.0)],
 [('털어넣으셈', 1.0)],
 [('근데', 1.0), ('똑같음', 1.0)],
 [('참치쌈싸서', 1.0), ('털어놓어야함', 1.0)],
 [('똑같다고', 1.0), ('레알', 1.0)],
 [('개충격', 1.0), ('시방', 1.0)],
 [('컴퍼스', 1.0)],
 [('잠깐만', 1.0)],
 [('레알똑같넼', 1.0)],
 [('시부랰', 

##### Run DTM model 

In [13]:
# take a long time to run....
NUM_TOPICS = 4

if not os.path.exists('kakao(DTM)_model'):
    dtm_model = ldaseqmodel.LdaSeqModel(corpus=corpus, id2word=dictionary, time_slice=time_slice, num_topics=NUM_TOPICS, passes=10)
    dtm_model.save('kakao(DTM)_model')
else:
    dtm_model = ldaseqmodel.LdaSeqModel.load('kakao(DTM)_model')

##### DTM 결과 보기

In [14]:
# 고정된 시간 내에서 전체 토픽보기
pprint(dtm_model.print_topics(time=1, top_terms=20))

[[('굿', 0.05831850538739241),
  ('지금', 0.012331524778776757),
  ('오', 0.0122166677634343),
  ('역시', 0.010455539788893783),
  ('내일', 0.009190718899343277),
  ('이미', 0.008990207437333388),
  ('왜', 0.007369435495622992),
  ('낼', 0.005757033715288104),
  ('야', 0.0051241059091660665),
  ('아직', 0.004872538898427595),
  ('시발', 0.0045842544466057865),
  ('그래도', 0.004528226788699147),
  ('사실', 0.004487026774860223),
  ('없음', 0.004297307967381041),
  ('하고', 0.004058918940601231),
  ('아니면', 0.0035801679800862318),
  ('가서', 0.0035703666039212257),
  ('빨리', 0.0035260000564760795),
  ('많이', 0.0034673740267698538),
  ('대단', 0.00334297411317377)],
 [('핳', 0.03620256743725127),
  ('그럼', 0.017891407390810514),
  ('나', 0.014458865122934468),
  ('나도', 0.01339997787202228),
  ('걍', 0.012045925968074068),
  ('닥쳐', 0.011429731540920987),
  ('나는', 0.009952254204843707),
  ('핳핳', 0.009887808067018595),
  ('뭐', 0.009682205321122319),
  ('Search', 0.008825802051635289),
  ('그래서', 0.007265567687011695),
  ('후발', 

In [15]:
# 고정된 토픽 내에서 전체 시간 변화 보기
dtm_model.print_topic_times(topic=0, top_terms=20)

[[('굿', 0.05805158291676767),
  ('오', 0.01257289751420625),
  ('지금', 0.012317538191021923),
  ('역시', 0.010446017713230754),
  ('내일', 0.009090115508344862),
  ('이미', 0.008981907367809726),
  ('왜', 0.007362543829053719),
  ('낼', 0.0057151185377723015),
  ('야', 0.00514056792096524),
  ('아직', 0.004834757285897352),
  ('그래도', 0.004495242502305918),
  ('사실', 0.0044546150915943895),
  ('시발', 0.004425428506800929),
  ('없음', 0.004283082178656709),
  ('하고', 0.004040773275026179),
  ('대단', 0.0036382537564949753),
  ('빨리', 0.0035261972049233747),
  ('가서', 0.0035020169139750867),
  ('아니면', 0.0034876003188684113),
  ('많이', 0.0034321705608634854)],
 [('굿', 0.05831850538739241),
  ('지금', 0.012331524778776757),
  ('오', 0.0122166677634343),
  ('역시', 0.010455539788893783),
  ('내일', 0.009190718899343277),
  ('이미', 0.008990207437333388),
  ('왜', 0.007369435495622992),
  ('낼', 0.005757033715288104),
  ('야', 0.0051241059091660665),
  ('아직', 0.004872538898427595),
  ('시발', 0.0045842544466057865),
  ('그래도', 0.

### 4. coherence score 계산 함수 만들기

#### topic 개수 지정하기.

In [16]:
start=4; end=9; step=1; # 4,5,6,7,8개의 토픽 개수 보고자하는 것
times=[0, 1, 2] # timeslice가 3개니까

#### DTM과 coherence score 계산하는 함수

In [ ]:
# 함수 작성.

# coherence score는 LDA 별로 계산됨
# LDA가 어떻게 있냐, 지금 DTM 1개인데 DTM 안에 LDA 3개 -> 토픽 개수만큼 있음
# 따라서 총 15개 (토픽 개수 5개 x LDA 3개)

def compute_coherence(dictionary, corpus, texts, times, start, end, step):
# 파라미터 : 토픽개수만큼 DTM 돌려야하니까 DTM에 들어있는 파라미터 모두 필요, 걔네들을 매번 time에 따라 계산해줘야하는
    model_list = [] # 토픽 개수 넣는 것에 따라 모델을 다 만들어야하니까 저장할 리스트
    coherence_score_list = []
    
    for num_topics in tqdm_notebook(range(start, end, step)):
        start_dtm = time() # 시작시간
        
        # DTM은 모델 하나하나 돌리는 데 시간이 아주 오래 걸려
        # 따라서 토픽을 줄 때마다 하나하나 저장하기 위한 규칙
        dtm_name = "kakao_dtm_model_" + str(num_topics)
        if os.path.exists(dtm_name):
            dtm_model = ldaseqmodel.LdaSeqModel.load(dtm_name)
        else: # 없으면 학습시켜줌
            dtm_model = ldaseqmodel.LdaSeqModel(corpus=corpus,
                                               id2word=dictionary,
                                               time_slice=times,
                                               num_topics=num_topics,
                                               passes=100) # 원래는 1000번 정도(하지만 어어어엄청 오래 걸리겠지)
            dtm_model.save(dtm_name)
        
        end_dtm = time()
        print('Elapsed Time for DTM in %d topics : %.2f sec' % (num_topics, (end_dtm - start_dtm)))
        
        topics_cs_list = []
        for time_slot in times: # 시간대 별 계신
            topics_dtm = dtm_model.dtm_coherence(time=time)
            cs = CoherenceModel(topics=topics_dtm, texts=texts, dictionary=dictionary, coherence='c_v')
            # texts는 tokenized된 토픽 모델링의 input으로 들어갔던 text임
            topic_cs_list.append(cs) #DTM 안의 토픽 개수 별 LDA 3개가 계산된 거 저장
        
        model_list.append(dtm_model)
        coherence_score_list.append(topic_cs_list)
        
        return model_list, coherence_score_list
    
# LDA 하나에 대해 coherence score가 계산됨 (LDA가 3개씩 있으니까 3개씩 5개 나옴)

In [ ]:
# 함수 실행.
model_list, coherence_scores = compute_coherence(dictionaray=dictionary,
                                                corpus=corpus,
                                                texts=tokenized_data,
                                                times=times,
                                                start=start, end=end, step=step)

##### 위 계산 결과 통째로 저장

In [ ]:
import pickle

with open("kakao_dtm_models.pk", 'wb') as f:
    pickle.dump(model_list, f)
    
with open("kakao_dtm_cs.pk", 'wb') as f:
    pickle.dump(coherence_scores, f)

In [ ]:
import pickle

with open("kakao_dtm_models.pk", 'rb') as f:
    model_list = pickle.load(f)
    
with open("kakao_dtm_cs.pk", 'rb') as f:
    coherence_scores = pickle.load(f)

### 5. Topic Coherence 변화 그래프 그리기

- Topic Coherence score를 왜 계산했었냐면, 우리가 원하는 건 best topic을 모르니까 (k를 모르니까) 4,5,6,7,8 중 좋은걸 고르려고
- topic 개수가 늘어나면 걸리는 시간도 조금씩 늘어남

In [ ]:
print(model_list, coherence_scores)

In [ ]:
pprint(coherence_scores)

In [ ]:
# 편한 연산을 위해 numpy array로 변경.
import numpy as np

# DTM 하나에 LDA마다 나온 거니까 평균을 때릴거야
# DTM 하나에 coherence score를 하나씩 주는거야
# topic best를 고르기 위해 이것들중 하나를 골라야하니까!
np_coherence_scores = np.array(coherence_scores)
np_cs_avg_list = np_coherence_scores.mean(axis=1) # 행 별로 평균내줘
print(np_cs_avg_list) # 5개 나옴

In [ ]:
# 그래프 출력을 위한 한글 글꼴 설정.

import matplotlib.pyplot as plt
import platform
from matplotlib import font_manager, rc
%matplotlib inline

path = "c:/Windows/Fonts/malgun.ttf"
if platform.system() == 'Darwin':
    rc('font', family='AppleGothic')
elif platform.system() == 'Windows':
    font_name = font_manager.FontProperties(fname=path).get_name()
    rc('font', family=font_name)
else:
    print('Unknown system... sorry~~~~')

In [ ]:
# 그래프 그리기
x = range(start, end, step)

plt.figure(figsize=(12, 10)) 
plt.plot(x, np_cs_avg_list, '-b')
plt.xlabel("토픽 수")
plt.ylabel("평균 일관성 점수 (avg of coherence score)")
plt.xticks(x)
plt.show()

##### 위에서 구한 일관성 점수 기준으로 최적 토픽 모델 결정

In [ ]:
# 최적 토픽 모델을 dtm_model로 할당
dtm_model = model_list[np.argmax(np_cs_avg_list)] #best coherence score를 가지는 model
# np.argmax() : numpy array 중에서 max값을 가지고있는 index를 뽑아줌
dtm_model.num_topics # 그래프에서 나온 best coherence score를 가지는 토픽 수가 제대로 나오는지 확인

##### DTM 결과 보기

In [ ]:
# 고정된 시간 내에서 전체 토픽보기
pprint(dtm_model.print_topics(time=1, top_terms=20))

In [ ]:
# 고정된 토픽 내에서 전체 시간 변화 보기
dtm_model.print_topic_times(topic=0, top_terms=20)

### 6. 문서별 토픽 비중 구하기

- 전체 data 중에서 이 토픽이 얼마나 중요, 전체 topic 중 얼마를 차지하냐
- 어떤 문서가 어떤 topic인지 다 계산해주고, 계산된 결과를 가지고 비중을 계산

> 1) 토픽 비중 계산을 위한 데이터 전처리

> 2) OOM 문제 해결을 위한 샘플링

> 3) DataFrame으로 만들어 excel 파일로 저장

In [16]:
# 문서별 시간 정보 추가.

# dtm에서 토픽이 변하는 걸 보고싶은거지 그래서 문서별로 시간 정보가 들어가야돼
# 그래야 시간대 별로 잡아서 계산해서 토픽 비중 구할 수 있으니까
doc_time = []
for doc_id in range(len(tokenized_data)):
    if doc_id < time_slice[0]: # 2018년도 데이터라는 것
        doc_time.append(0) # time_slice를 넣어주는거지
    elif doc_id < time_slice[0] + time_slice[1]: # time_slice[0] < doc_id < time_slice[1]
        doc_time.append(1)
    else:
        doc_time.append(2)
        
print(len(doc_time)) # data 개수만큼 나오면 잘 된 것

22914


In [17]:
# time_slot 별로 1000개씩만 뽑기.
# 원래는 비율대로 sampling 뽑는게 좋지만 편의상 time_slice별로 1000개씩 뽑음
sample_idx1 = np.random.randint(0, time_slice[0], size=1000)
sample_idx2 = np.random.randint(time_slice[0], time_slice[0] + time_slice[1], size=1000)
sample_idx3 = np.random.randint(time_slice[0] + time_slice[1], time_slice[0] + time_slice[1] + time_slice[2],
                                size=1000)

print(sample_idx1[:10])
print(sample_idx2[:10])
print(sample_idx3[-10:])

[5871 5260 5486 3436 4989 4514 1831 3718 3527 4733]
[17744 13312  9003 12080 16970  9118 10432 18792  7238  9906]
[21763 22126 20013 20030 21825 21419 21617 21620 20938 22833]


In [18]:
# OOM(Out-of-Memory) 문제로 인해 1000개씩 샘플링 하여 실행.

processing_docs = []
processing_time = []

sample_indices = np.concatenate((sample_idx1, sample_idx2, sample_idx3))
# concatenate : 옆으로 붙여줌

for idx in sample_indices:
    processing_docs.append(tokenized_data[idx])
    processing_time.append(doc_time[idx])
    
print(len(processing_docs))
print(len(processing_time))

3000
3000


In [ ]:
# 문서별로 각 토픽에 속할 확률값을 계산.

# 비중 계산 어떻게? -> 문서를 DTM 모델에 넣어서 "이거 몇번 토픽이야?" 이거를 뽑아주고 다 누적해서 계산하면 돼
doc_dist = [] # 누적해서 모아주는 애
for doc_id in sample_indices:
    doc_list.append(dtm_model.doc_topics(doc_id)) # dtm_model.doc_topics() : doc_id 넣어주면 얘가 어떤 topic에 들어갈건지 정보가 나옴
    
doc_dist[:10] # 확률값이 쭉 나올 것임

In [ ]:
# 원하는 형태의 DataFrame으로 바꿔주기 위한 전처리.

doc_dist = np.array(doc_dist)
doc_topic_dist =doc_dist.T
doc_topic_dist.shape

In [ ]:
# 데이터값 확인.

NUM_TOPICS = dtm_model.num_topics

for i in range(NUM_TOPICS):
    print("%f" % doc_dist[0][i], end=", ")

print()

for i in range(NUM_TOPICS):
    print("%f" % doc_topic_dist[i][0], end=", ")

In [ ]:
doc_topic_dist[0].shape

In [ ]:
# DataFrame으로 만들고, excel파일로 저장.

dtm_df = pd.DataFrame({"Time" : processing_time, "Topic0" : doc_topic_dist[0], "Topic1" : doc_topic_dist[1],
                      "Topic2" : doc_topic_dist[2], "Topic3" : doc_topic_dist[3], "Topic4" : doc_topic_dist[4],
                       "Topic5" : doc_topic_dist[5], "Topic6" : doc_topic_dist[6], "Topic7" : doc_topic_dist[7]
                      })

print(dtm_df.head())
dtm_df.to_excel("kakao_dtm_8topic_dist.xlsx")

### 7. 시간별 토픽비중 변화 추이

> 1) 토픽 비중 변화를 보기 위한 데이터 전처리

> 2) 시간대별 토픽 비중 평균 계산

##### timeslot별로 평균 구하기

In [ ]:
# groupby 오브젝트 생성하기
doctopic_timeslot = dtm_df.groupby('Time')
doctopc_timeslot.groups
# 1000개씩 0,1,2 나오겠지

In [ ]:
# mean() 함수 이용해서 Topic0~TopicN까지의 모든 열 평균 구하기
timeslot_avg = doctopic_timeslot.mean() # 시간대별 토픽비중 보고싶어서 평균!
timeslot_avg = timeslot_avg.reset_index()
timeslot_avg

In [ ]:
# 시기별 모든 토픽 값들의 합은 1이다.
timeslot_avg.sum(axis=1) # 행 별로

### 8. 6~7번 결과로 그래프 그리기

#### 시간별 토픽 비중변화 추이

In [ ]:
# figure 하나당 4개씩 그래프 그리기

plt.figure()
plt.figure(figsize=(16, 12)) 
plt.plot(timeslot_avg['Time'], timeslot_avg['Topic0'], 'b*--', label='토픽0')
plt.plot(timeslot_avg['Time'], timeslot_avg['Topic1'], 'rs--', label='토픽1')
plt.plot(timeslot_avg['Time'], timeslot_avg['Topic2'], 'g^--', label='토픽2')
plt.plot(timeslot_avg['Time'], timeslot_avg['Topic3'], 'y--', label='토픽3')
plt.plot(timeslot_avg['Time'], timeslot_avg['Topic3'], 'p--', label='토픽4')
plt.plot(timeslot_avg['Time'], timeslot_avg['Topic3'], 'm--', label='토픽5')
plt.plot(timeslot_avg['Time'], timeslot_avg['Topic3'], 'k--', label='토픽6')
plt.plot(timeslot_avg['Time'], timeslot_avg['Topic3'], 'c--', label='토픽7')

#그래프에서 값 표시하는 annotate 방법 찾기
# 토픽들 결과를 그래프 위에 찍어주려고
for topic_id in ['Topic0', 'Topic1', 'Topic2', 'Topic3']:
    for x, y in zip(range(len(time_slice)), timeslot_avg[topic_id]):
        plt.annotate("%.4f"%y, (x, y), textcoords="offset points", xytext=(0,10), ha='center')

plt.title("8개의 대화 토픽", fontsize=14)
plt.xlabel("대화 시기", fontsize=13)
plt.ylabel("토픽의 비중 (%)", fontsize=13)
plt.xticks(timeslot_avg['Time'], ['2017년', '2018년', '2019년'])
plt.ylim([0.08, 0.15])
plt.legend(loc="best")
plt.savefig("kakao_topic_distribution_graph.png")
plt.show()

#### 시간별 단어의 토픽 비중변화 추이

In [ ]:
from collections import defaultdict
# 단어의 확률 변화를 보고 싶은 topic 번호 

topic_id = 7 # 내가 보고싶은 topic 번호 정해줘

# 해당 토픽에서 포함할 단어목록 결정
topic_info = dtm_model.print_topic_times(topic=topic_id, top_terms=)
topic_dict = defaultdict(list)

#time별 특정 topic(topic_id)의 단어별 확률값.
for dist_inTime in topic_info:
    #time별로 단어와 확률값 저장.
    for word, prob in dist_inTime:
        topic_dict[word].append(prob)

In [ ]:
print(topic_dict)

In [ ]:
# 확률값 추이를 보고싶은 단어 목록
# 위에서 정해준 topic 번호에 대해서 그 단어의 확률값 추이를 나타내는 거임!!
keywords = ["야", "헐"] # 7번 topic에서의 "야", "헐" 단어의 확률값 추이

# time_slot
t = [i for i in range(len(time_slice))]

plt.figure(figsize=(16, 12))
for keyword in keywords:
    plt.plot(t, topic_dict[keyword], label=keyword)
    
    for x, y in zip(t, topic_dict[keyword]):
         plt.annotate("%.4f"%y, (x, y), textcoords="offset points", xytext=(0,10), ha='center')

plt.title("시간별 단어의 토픽비중 변화")
plt.xlabel("시간")
plt.ylabel("토픽 비중")
plt.legend(loc='best')
plt.savefig("dtm_kakao_topic_words-dist_overtime.png")
plt.show()